# Random Forest

**Random Forest** или Случайный лес - модель, используемая как для регрессии, так и для классификации. Алгоритм основан на другом алгоритме - Decision Tree или Дерево решений, т.е. деревья используются для создания леса. Термин **"случайный"** говорит о том, что каждое дерево построено на основе случайной подвыборки данных.

Для начала разберемся в том, как работает алгоритм Decision Tree.

## Decision Tree

Деревья решений состоят из двух элементов: узлы и ветви. Сам алгоритм основан на рекурсивном разбиении выборки по признаку, который даёт наибольший прирост информации (Gain), т.е. наилучшим образом разделяет выборку на данный момент. Разбиение будет происходить до тех пор, пока мы не дойдём до листьев - узлов, содержащих только экземпляры одного класса (pure, чистых). Пример дерева решений изображен на рисунке ниже.

<img src="images/decision_tree.svg" width=70%/>

Как видно из рисунка, существует несколько типов узлов:

1. **Корневой узел** &mdash; узел на вершине дерева. Содержит признак, который лучше всего делит данные (единственный признак, который классифицирует целевую переменную наиболее точно). Имеет 2 дочерних узла.
2. **Узел** &mdash; внутренний узел дерева, узел проверки. Каждый такой узел имеет 2 дочерних узла.
3. **Лист** &mdash; конечный узел дерева, узел решения. Не имеет потомков.

Для оценки качества разбиения выборки по признаку используется **энтропия Шеннона**. Энтропия рассматривается как мера неоднородности подмножества по представленным в нем классам. Если классы представлены в равных долях, неопределенность классификации наибольшая, то и энтропия тоже максимальная. Логарифм от единицы будет обращать энтропию в ноль, если все примеры узла относятся к одному классу. Может быть расчитана по следующей формуле:
$$H(T) = -\sum_{i=1}^n p_i log_2 p_i = -\sum_{i=1}^n \frac{N_i}{N} log_2 \left(\frac{N_i}{N}\right),$$
где $T$ - вектор значений целевой переменной (target), $n$ — число классов в исходном подмножестве, $N_i$ — число экземпляров i-го класса в подмножестве, $N$ — общее число экземпляров в подмножестве.

In [23]:
import numpy as np
from collections import Counter

In [91]:
def entropy(t):
    counts = np.bincount(t)
    probability = counts / len(t)
    
    entropy = 0
    for prob in probability:
        if prob > 0:
            entropy += prob * np.log2(prob)
    return -entropy

In [92]:
s = [0, 0, 0, 0, 0, 0, 0, 1, 1, 1]
print(f'Entropy: {np.round(entropy(s), 5)}')

Entropy: 0.88129


На практике, однако, говорят не об энтропии, а о величине, обратной ей, которая называется **Приростом информации (Inforamtion Gain, IG)**. Тогда лучшим атрибутом разбиения будет тот, который обеспечит максимальный прирост информации результирующего узла относительно исходного. 
Прирост информации представляет собой разницу между энтропией исходной выборки и взвешенным средним суммы всех значений энтропии для разделённых подмножеств. Чем выше значение информационного прироста, тем лучше принятое решение. Может быть расчитан по следующей формуле:
$$IG(T, A) = H(T) - H(T|A)$$
$$IG(T, A) = H(T) - \sum_{v \in Values(A)}\frac{|T_{v}|}{|T|}H(T_{v}),$$
где $T$ - вектор значений целевой переменной (target), $A$ - признак, по которому было совершено разбиение, $v$ - каждое уникальное значение признака $A$.

Если объяснить немного по-другому, мы находим энтропию каждого набора после разбиения, взвешиваем ее по количеству элементов в каждом разбиении, затем вычитаем из текущей энтропии. Если результат положительный, значит, мы снизили энтропию при разбиении. Чем выше результат, тем больше мы понизили энтропию.

In [26]:
def information_gain(parent, left, right):
    num_left = len(left) / len(parent)
    num_right = len(right) / len(parent)
    
    gain = entropy(parent) - (num_left * entropy(left) + num_right * entropy(right))
    return gain

In [27]:
parent = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1]
left_child = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1]
right_child = [0, 0, 0, 0, 1, 1, 1, 1]
print(f'Information gain: {np.round(information_gain(parent, left_child, right_child), 5)}')

Information gain: 0.18094


## Реализация Decision Tree на Python
Для реализации необходимы 2 класса:
* Node - реализует один узел дерева решений
* DecisionTree - реализует алгоритм дерева решений

### Класс, реализующий один узел дерева
Класс, необходимый для хранения данных о признаке(feature) и пороговом значении(threshold), по которым производилось разбиение выборки, поддеревья левое(data_left) и правое(data_right), информационном приросте(gain) и значении классификации(value), которое устанавливается только для листовых узлов.

In [28]:
class Node:
    def __init__(self, feature=None, threshold=None, data_left=None, data_right=None, gain=None, value=None):
        self.feature = feature
        self.threshold = threshold
        self.data_left = data_left
        self.data_right = data_right
        self.gain = gain
        self.value = value

### Класс, реализующий дерево решений
Класс содержит следующие методы:
1. Конструктор **\_\_init__**, в котором задаются значения для min_samples_split и max_depth. Это гиперпараметры. Первый используется для указания минимального количества образцов, необходимых для разбиения узла, а второй задает максимальную глубину дерева. Оба параметра используются в рекурсивных функциях в качестве условий выхода.
1. Метод **\_entropy(t)** вычисляет энтропию входного вектора t
1. Метод **\_information_gain(parent, left, right)** вычисляет значение информационного прироста при разделении родительского и двух дочерних векторов.
1. Метод **\_best_split(X, y)** вычисляет наилучшие параметры разбиения для входных признаков X и целевой переменной y. Это делается путем итерации по каждому столбцу в X и по каждому пороговому значению в каждом столбце, чтобы найти оптимальное разбиение с использованием информационного прироста.
1. Метод **\_build(X, y, depth)** рекурсивно строит дерево решений до достижения критериев останова (гиперпараметры в конструкторе).
1. Метод **fit(X, y)** вызывает функцию **\_build()** и сохраняет построенное дерево
1. Метод **\_predict(x)** обходит дерево для классификации одного экземпляра
1. Метод **predict(X)** применяет функцию **\_predict()** к каждому экземпляру в матрице X

In [93]:
class DecisionTree:
    '''
    Класс, реализующий классификатор на основе алгоритма дерева решений
    '''
    def __init__(self, min_samples_split=2, max_depth=5):
        self.min_samples_split = min_samples_split
        self.max_depth = max_depth
        self.root = None
    
    @staticmethod
    def _entropy(t):
        '''
        Вспомогательная функция, вычисляет энтропию массива целочисленных значений
        '''
        counts = np.bincount(np.array(t, dtype=np.int))
        probability = counts / len(t)
    
        entropy = 0
        for prob in probability:
            if prob > 0:
                entropy += prob * np.log2(prob)
        return -entropy
    
    def _information_gain(self, parent, left, right):
        '''
        Вспомогательная функция, вычисляет информационный прирост от родителя и двух узлов-потомков
        '''
        num_left = len(left) / len(parent)
        num_right = len(right) / len(parent)
    
        gain = self._entropy(parent) - (num_left * self._entropy(left) + num_right * self._entropy(right))
        return gain
    
    def _best_split(self, X, y):
        '''
        Вспомогательная функция, вычисляет лучшее разбиение для заданных признаков и меток
        '''
        best_split = {}
        best_info_gain = -1
        n_rows, n_cols = X.shape
        
        # for every feature
        for f_idx in range(n_cols):
            X_curr = X[:, f_idx]
            # for every unuque value of feature
            for threshold in np.unique(X_curr):
                df = np.concatenate((X, y.reshape(1, -1).T), axis=1)
                df_left = np.array([row for row in df if row[f_idx] <= threshold])
                df_right = np.array([row for row in df if row[f_idx] > threshold])
            
                if len(df_left) > 0 and len(df_right) > 0:
                    y = df[:, -1]
                    y_left = df_left[:, -1]
                    y_right = df_right[:, -1]
                
                    gain = self._information_gain(y, y_left, y_right)
                    if gain > best_info_gain:
                        best_split = {
                            'feature_index': f_idx,
                            'threshold': threshold,
                            'df_left': df_left,
                            'df_right': df_right,
                            'gain': gain
                        }
                        best_info_gain = gain
        return best_split
    
    def _build(self, X, y, depth=0):
        '''
        Вспомогательная рекурсивная функция, используется для построения дерева решений по входным данным
        '''
        n_rows, n_cols = X.shape
        
        if n_rows >= self.min_samples_split and depth <= self.max_depth:
            best = self._best_split(X, y)
            if best['gain'] > 0: # impure split
                left = self._build(
                    X=best['df_left'][:, :-1], 
                    y=best['df_left'][:, -1], 
                    depth=depth + 1
                )
                right = self._build(
                    X=best['df_right'][:, :-1], 
                    y=best['df_right'][:, -1], 
                    depth=depth + 1
                )
                return Node(
                    feature=best['feature_index'], 
                    threshold=best['threshold'], 
                    data_left=left, 
                    data_right=right, 
                    gain=best['gain']
                )
        # лист (pure), value - самое часто встречающееся значение в y
        return Node(
            value=int(Counter(y).most_common(1)[0][0])
        )
    
    def fit(self, X, y):
        '''
        Функция для обучения классификатора дерева решений
        '''
        self.root = self._build(X, y)
    
    def _predict(self, x, tree):
        '''
        Вспомогательная рекурсивная функция для предсказания одного значения (обход дерева)
        '''
        # leaf
        if tree.value != None:
            return tree.value
        feature_value = x[tree.feature]
        
        # go left
        if feature_value <= tree.threshold:
            return self._predict(x=x, tree=tree.data_left)
        
        # go right
        if feature_value > tree.threshold:
            return self._predict(x=x, tree=tree.data_right)
        
    def predict(self, X):
        '''
        Функция для предсказания значений меток
        '''
        return np.array([self._predict(x, self.root) for x in X])

## Тестирование

In [94]:
from sklearn.datasets import load_iris

iris = load_iris()

X = iris['data']
y = iris['target']

In [95]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [96]:
model = DecisionTree()

In [97]:
model.fit(X_train, y_train)

In [98]:
y_pred = model.predict(X_test)

In [99]:
y_pred

array([1, 0, 2, 1, 1, 0, 1, 2, 1, 1, 2, 0, 0, 0, 0, 1, 2, 1, 1, 2, 0, 2,
       0, 2, 2, 2, 2, 2, 0, 0])

In [100]:
y_test

array([1, 0, 2, 1, 1, 0, 1, 2, 1, 1, 2, 0, 0, 0, 0, 1, 2, 1, 1, 2, 0, 2,
       0, 2, 2, 2, 2, 2, 0, 0])

In [101]:
from sklearn.metrics import accuracy_score

accuracy_score(y_test, y_pred)

1.0

Сравним построенную модель с классификатором из библиотеки Scikit-Learn DecisionTreeClassifier.

In [102]:
from sklearn.tree import DecisionTreeClassifier

sk_model = DecisionTreeClassifier()
sk_model.fit(X_train, y_train)
sk_y_pred = sk_model.predict(X_test)

accuracy_score(y_test, sk_y_pred)

1.0

## Random Forest

Алгоритм Случайного леса состоит из трёх компонентов:
$$Random Forest = Decision Tree + Bagging + Random Subspace$$

<img src="images/comparison.png" width=70%/>

## Реализация Random Forest на Python

Класс содержит следующие методы:

1. **\_\_init__()** - конструктор, хранит значения гиперпараметров для количества деревьев в лесе, минимального количества элементовв разбиении дерева и максимальной глубины. В нём также будут храниться индивидуально обученные деревья решений после обучения модели.
1. **\_sample(X, y)** применяет bootstrap-выборку к входным признакам и входной целевой переменной
1. **fit(X, y)** метод для обучения модели классификатора
1. **predict(X)** делает прогнозы с помощью отдельных деревьев решений, а затем применяет мажоритарное голосование для окончательного прогноза

In [106]:
class RandomForest:
    '''
    Класс, реализующий алгоритм Слечайного леса
    '''
    def __init__(self, num_trees=25, min_samples_split=2, max_depth=5):
        self.num_trees = num_trees
        self.min_samples_split = min_samples_split
        self.max_depth = max_depth
        # Храним все построенные деревья
        self.decision_trees = []

    @staticmethod
    def _sample(X, y):
        '''
        Вспомогательная функция для создания bootstrap-выборки
        '''
        n_rows, n_cols = X.shape
        # Sample with replacement
        samples = np.random.choice(a=n_rows, size=n_rows, replace=True)
        return X[samples], y[samples]
    
    def fit(self, X, y):
        '''
        Обучает классификатор Случайный лес
        '''
        # Очистка списка деревьев при повторном запуске обучения
        if len(self.decision_trees) > 0:
            self.decision_trees = []
            
        # Построение каждого дерева леса
        num_built = 0
        while num_built < self.num_trees:
            try:
                clf = DecisionTree(
                    min_samples_split=self.min_samples_split,
                    max_depth=self.max_depth
                )
                # Получаем подвыборку
                _X, _y = self._sample(X, y)
                # Обучаем
                clf.fit(_X, _y)
                # Сохраняем классификатор
                self.decision_trees.append(clf)
                num_built += 1
            except Exception as e:
                continue
        
    def predict(self, X):
        '''
        Предсказывает метку класса для новых объектов
        '''
        # Сделать предсказание каждым деревом в лесу
        y = []
        for tree in self.decision_trees:
            y.append(tree.predict(X))
        
        # Решейп для поиска наиболее частого значения
        y = np.swapaxes(a=y, axis1=0, axis2=1)
        
        # Используем голосование большинством голосов для финального предсказания
        predictions = []
        for preds in y:
            counter = Counter(x)
            predictions.append(counter.most_common(1)[0][0])
        return predictions